In [1]:
import sys
sys.path.append('..')

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.pyplot import imshow
from tqdm import tqdm
import torch
from torch.nn import Module
from torchsummary import summary

from collections import namedtuple, defaultdict
from pathlib import Path
import time

%matplotlib inline
rcParams['figure.figsize'] = (10, 15)

In [3]:
from src.constructor.config_structure import TrainConfigParams
from src.registry import TASKS
from src.constructor.data import create_dataset
from src.constructor.trainer import download_s3_artifact
from train import load_config

../src/data/datasets/classification.py:130: DeprecationWarning: invalid escape sequence \d
  label = list(map(int, re.findall('\d+', label)))


In [4]:
def load_hparams(config_path):
    hparams = load_config(config_path)
    hparams = TrainConfigParams(**hparams)
    
    return hparams

/home/ubuntu/miniconda3/envs/torchok/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
config_path = '../configs/siloiz_pairwise_xbm_resnet50_adam_512d.yml'
config = load_hparams(config_path)

In [6]:
# Prevent XBM from loading
config.task.params['xbm_params'] = None

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
checkpoint = torch.load(download_s3_artifact('s3://ml-fips/mlruns/eedfa4c178bd44f8bc6c44cdcc0f4814/artifacts/last.ckpt', '/tmp'), map_location=device)
model = TASKS.get(config.task.name)(config)
model = model.to(device)
model.load_state_dict(checkpoint['state_dict'])
model.eval();

<All keys matched successfully>


In [9]:
traced = torch.jit.trace(model.forward, torch.rand(4, 3, 224, 224, device=device))

../src/models/backbones/resnet.py:597: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.size(1) == self.encoder_channels[i]:


In [10]:
traced.save('/mnt/ext1/fips-benchmark/models/siloiz_contrastive_xbm_resnet50_adam_512d.pt')